In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub


import torch
from tqdm.notebook import tqdm
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
!pip install numba





In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Add and preprocess data


Thêm và xử lý dữ liệu theo format của thầy


In [ ]:
with open('/content/drive/MyDrive/ColabNotebooks/TextClassification/product_train_0.txt', 'r', encoding='utf-8') as file:
    data = file.read().split('\n')
    raw_df = pd.DataFrame([line.split(' ', 1) for line in data], columns=['Label', 'Description'])
#remove prefix
raw_df['Label'] = raw_df['Label'].str.replace('__label__', '').str.replace('_', ' ')
raw_df = raw_df.drop(raw_df.index[-1])
raw_df.describe()


,Label,Description
count,14976,14976
unique,1971,14966
top,"Bao đựng, ốp lưng điện thoại",Kệ Tivi KDCN39
freq,342,2


Thêm và xử lý dữ liệu theo data crawl (đã xử lý)

In [ ]:
raw_df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/TextClassification/filtered_50_data.csv')

Thêm và xử lý dữ liệu theo data của long


In [ ]:
raw_df = pd.read_excel('/content/drive/MyDrive/ColabNotebooks/TextClassification/label_crawl_from_tiki.xlsx')
raw_df = raw_df[['Label', 'Description']]

Thêm và xử lý dữ liệu merged từ cả 2

In [ ]:
raw_df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/TextClassification/merged_data.csv')

In [ ]:
raw_df.head()

,Label,Description,label_index,data_type
0,Ghế văn phòng,Ghế HQ - HK095,523,train
1,Thước dây,30m Thước dây làm bằng sợi thủy tinh TOTAL TMT...,1460,train
2,Cưa tay,Cưa gỗ cầm tay cán lớn Asaki AK - 8657,357,train
3,"Váy, đầm",Đầm ren cổ V tay ngắn cao cấp,1648,val
4,"Dụng cụ đo, kiểm tra khác",Máy đo độ đồng tâm HANN YAN 6401D1,488,train


In [ ]:
raw_df['Label'].value_counts()

Bao đựng, ốp lưng điện thoại    342
Bàn ghế phòng khách             187
Bộ bàn ghế cafe                 175
Váy, đầm                        158
Bộ bàn ghế ( phòng ăn, bếp )    135
                               ... 
Tời kéo                           1
Van công nghiệp khác              1
Máy in khổ lớn                    1
Bể cá                             1
Bánh pía                          1
Name: Label, Length: 1971, dtype: int64

Lọc ra các entry bị duplicate (cho cái của thanh)
- Bỏ bước này


In [ ]:
# Use value_counts() to count the occurrences of each value in the column
value_counts = raw_df['Description'].value_counts()

# Filter values that appear more than once
duplicated_values = value_counts[value_counts > 1]

raw_df[raw_df['Description'].isin(duplicated_values.index)].sort_values(by='Description')

# # Drop duplicate rows, keeping the first occurrence
raw_df = raw_df.drop_duplicates(subset='Description', keep='first')

# raw_df[raw_df['Description'].isin(duplicated_values.index)].sort_values(by='Description')


Thêm code chỉ lấy 50 bản ghi (cho data crawl)
- Bỏ bước này

In [ ]:
# Lấy danh sách các nhãn duy nhất
possible_labels = raw_df['Label'].unique()

# Khởi tạo một DataFrame mới để lưu kết quả
filtered_df = pd.DataFrame()

dataframes=[]
# Duyệt qua mỗi nhãn và lấy 50 bản ghi đầu tiên cho mỗi nhãn
for label in possible_labels:
    label_data = raw_df[raw_df['Label'] == label][:50]
    dataframes.append(label_data)

filtered_df = pd.concat(dataframes, ignore_index=True)

# Kết quả là DataFrame chứa 50 bản ghi đầu tiên của mỗi nhãn
raw_df = filtered_df

Lưu data sau lọc ra csv
- Bỏ bước này

In [ ]:
raw_df.to_csv('/content/drive/MyDrive/ColabNotebooks/TextClassification/filtered_data.csv', index = False)

Đánh index cho từng label

In [ ]:
possible_labels = raw_df.Label.unique()
possible_labels = sorted(possible_labels)

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Access point (Wifi)': 0,
 'Ampe kìm': 1,
 'Amplifier': 2,
 'Amplifier Hi-End': 3,
 'An toàn cho bé khi đi ô tô': 4,
 'An toàn cho trẻ': 5,
 'Ba lô học sinh': 6,
 'Ba lô leo núi, du lịch': 7,
 'Baga xe máy': 8,
 'Balo thời trang': 9,
 'Bao bì thực phẩm': 10,
 'Bao cao su': 11,
 'Bao cố định chấn thương': 12,
 'Bao đựng, ốp lưng điện thoại': 13,
 'Barie tự động': 14,
 'Bia': 15,
 'Biến tần': 16,
 'Biến áp máy cấp nguồn đa năng': 17,
 'Biển báo giao thông': 18,
 'Biển chỉ': 19,
 'Biển, bảng, đèn thông báo ( Led )': 20,
 'Block điều hòa': 21,
 'Bluetooth USB Adapter': 22,
 'Board mạch điều khiển': 23,
 'Body liền cho bé': 24,
 'Boot nữ': 25,
 'Bu lông, tắc kê': 26,
 'Buồng tắm đa năng': 27,
 'Buộc tóc': 28,
 'Buộc tóc trẻ em': 29,
 'Bài vị thờ': 30,
 'Bàn Billiard (Bi A)': 31,
 'Bàn cafe ( café)': 32,
 'Bàn chơi game - Bàn phòng Game': 33,
 'Bàn chải đánh răng': 34,
 'Bàn chế biến thực phẩm': 35,
 'Bàn chụp sản phẩm': 36,
 'Bàn ghế học tập và giảng dạy': 37,
 'Bàn ghế nhà hàng, khách sạn

In [ ]:
len(label_dict)

1971

In [ ]:
raw_df['label_index'] = raw_df.Label.replace(label_dict)

In [ ]:
len(raw_df)

In [ ]:
raw_df.head()

,Label,Description,label_index,data_type
0,Microphone,Microphone Led RGB TSP2012 kết nối USB cho máy...,785,val
1,Microphone,Microphone Razer Seiren Mini - Hàng Chính Hãng,785,val
2,Microphone,Wireless Lavalier Microphone Clip-on Phone Wir...,785,val
3,Microphone,Microphone Kingston HyperX QuadCast S RGB - Hà...,785,val
4,Microphone,Microphone cài áo Rode Lavalier Go - Hàng Chín...,785,val


In [ ]:
raw_df.tail()

,Label,Description,label_index,data_type
98545,Cửa kéo,Rèm Cửa Sổ In Họa Tiết Hoạt Hình Dễ Thương Xin...,395,val
98546,Cửa kéo,Mành Rèm cửa - ngang 1m x cao 1m3,395,val
98547,Cửa kéo,Rèm cửa cản nắng polyester cao cấp màu xám môn,395,val
98548,Cửa kéo,Rèm dán cửa không cần thanh treo (size 1m x 2m...,395,val
98549,Cửa kéo,Rèm Cửa Rido Chống Nắng - Ngăn Phòng Móc Thẳng...,395,val


# Train model

Tách tập train và test https://stackoverflow.com/questions/60636444/what-is-the-difference-between-x-test-x-train-y-test-y-train-in-sklearn

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split (raw_df.index.values,
                                                  raw_df.label_index.values,
                                                  test_size=0.85,
                                                  random_state=42,
                                                  # stratify=raw_df.label_index.values
                                                   )

raw_df['data_type'] = ['not_set']*raw_df.shape[0]

raw_df.loc[X_train, 'data_type'] = 'train'
raw_df.loc[X_val, 'data_type'] = 'val'

raw_df.groupby(['Label', 'label_index', 'data_type']).count()

BertTokenizer and Encoding the Data

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encoded_data_train = tokenizer.batch_encode_plus(
    raw_df[raw_df.data_type=='train'].Description.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    raw_df[raw_df.data_type=='val'].Description.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(raw_df[raw_df.data_type=='train'].label_index.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(raw_df[raw_df.data_type=='val'].label_index.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

BERT Pre_trained model (train tiếp thì không cần)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Data loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32
dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

Optimizer and scheduler (train tiếp thì không cần)


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

epochs = 7

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Performance metrics


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label_index in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label_index]
        y_true = labels_flat[labels_flat==label_index]
        print(f'Class: {label_dict_inverse[label_index]}')
        print(f'Accuracy: {len(y_preds[y_preds==label_index])}/{len(y_true)}\n')

Training loop


In [ ]:
import torch

# Kiểm tra xem GPU có sẵn không
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

GPU is available.


In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/'):
    os.makedirs('/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/')

In [ ]:
import random
import torch
torch.cuda.empty_cache()


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        model.to(device)

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

# Evaluate

In [ ]:
import random
import torch
torch.cuda.empty_cache()


seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

Evaluating model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

checkpoint = torch.load('/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/checkpoint_epoch_15.pth')


model.load_state_dict(checkpoint['model_state_dict'])

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Streaming output truncated to the last 5000 lines.
Accuracy: 3/8

Class: Cà phê hạt
Accuracy: 0/6

Class: Cà phê phin
Accuracy: 16/18

Class: Cá
Accuracy: 2/2

Class: Cá khô
Accuracy: 2/3

Class: Các linh kiện khác (máy tính)
Accuracy: 0/4

Class: Các loại kem phấn cho bé
Accuracy: 2/7

Class: Các loại máy khác
Accuracy: 0/1

Class: Các loại móc
Accuracy: 30/33

Class: Các loại nước hoa khác
Accuracy: 0/2

Class: Các loại phụ kiện ô tô khác
Accuracy: 34/45

Class: Các loại vật liệu hoàn thiện khác
Accuracy: 0/9

Class: Các món ăn khác
Accuracy: 0/1

Class: Các phụ kiện điện thoại khác
Accuracy: 0/1

Class: Các phụ tùng xe máy khác
Accuracy: 14/31

Class: Các phụ tùng xe đạp khác
Accuracy: 1/1

Class: Các sản phẩm cơ khí khác
Accuracy: 0/1

Class: Các sản phẩm khác
Accuracy: 0/12

Class: Các sản phẩm nhựa khác
Accuracy: 0/4

Class: Các thiết bị tản nhiệt
Accuracy: 2/3

Class: Các vật tư phụ
Accuracy: 0/2

Class: Các đồ dùng bơi lội khác
Accuracy: 0/4

Class: Cáp kết nối tivi
Accuracy: 1

In [ ]:
from sklearn.metrics import accuracy_score


# Calculate predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_vals, predicted_labels)

# Convert accuracy to percentage
accuracy_percentage = accuracy * 100

print(f'Accuracy: {accuracy_percentage:.2f}%')


Accuracy: 69.86%


# Load fineTuned model's state and continue training process

Thêm code để tiếp tục train, save state của mô hình và state của optimizer

In [ ]:
import random
import torch
from transformers import AdamW, get_linear_schedule_with_warmup

torch.cuda.empty_cache()

device = torch.device('cuda')
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

checkpoint = torch.load('/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/checkpoint_epoch_8.pth')
model.load_state_dict(checkpoint['model_state_dict'])

epochs = 7

model.to(device)

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

for epoch in tqdm(range(16, 16+epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        model.to(device)

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'epoch':epoch
    }, f'/content/drive/MyDrive/ColabNotebooks/TextClassification/fineTuned/checkpoint_epoch_{epoch}.pth')


    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')